In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import pandas as pd
file_path = 'gs://loan-data-bucket-anmol/cleaned_loan_data.csv'
df = pd.read_csv(file_path,low_memory=False)
df

,loan_amnt,term,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,desc,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag,fico_score
0,3600.0,36 months,13.99,123.03,MORTGAGE,55000.0,Not Verified,0,n,NaN,...,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,Cash,N,677.0
1,24700.0,36 months,11.99,820.28,MORTGAGE,65000.0,Not Verified,0,n,NaN,...,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,Cash,N,717.0
2,20000.0,60 months,10.78,432.66,MORTGAGE,63000.0,Not Verified,0,n,NaN,...,0.0,0.0,218418.0,18696.0,6200.0,14877.0,N,Cash,N,697.0
3,10400.0,60 months,22.45,289.91,MORTGAGE,104433.0,Source Verified,0,n,NaN,...,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,Cash,N,697.0
4,11950.0,36 months,13.44,405.18,RENT,34000.0,Source Verified,0,n,NaN,...,0.0,0.0,16900.0,12798.0,9400.0,4000.0,N,Cash,N,692.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345305,18000.0,60 months,9.49,377.95,OWN,130000.0,Not Verified,0,n,NaN,...,0.0,0.0,275356.0,54349.0,13100.0,77756.0,N,Cash,N,737.0
1345306,29400.0,60 months,13.99,683.94,MORTGAGE,180792.0,Not Verified,0,n,NaN,...,0.0,0.0,719056.0,148305.0,56500.0,95702.0,N,Cash,N,707.0
1345307,32000.0,60 months,14.49,752.74,MORTGAGE,157000.0,Source Verified,1,n,NaN,...,0.0,0.0,524379.0,122872.0,15800.0,23879.0,N,Cash,N,737.0
1345308,16000.0,60 months,12.79,362.34,RENT,150000.0,Not Verified,0,n,NaN,...,3.0,0.0,87473.0,65797.0,10100.0,73473.0,N,Cash,N,667.0


In [ ]:
#Picking the columns that are useful inn our model and target is the one we are predicting
selected_features=[
    'loan_amnt',
    'term',
    'int_rate',
    'installment',
    'annual_inc',
    'dti',
    'fico_range_high',
    'purpose',
    'home_ownership'
]

target  = 'loan_status'

In [ ]:
# create a smaller/dummy table with the columns we just need + target
data_model = df[selected_features+ [target]].copy()

#clean term column, it has 36 and 60 months written, convert the text to numbers 36 and 60
data_model["term"] = data_model["term"].str.extract('(\d+)').astype(int)
#.str.extract('(\d+)')-
#From each string, extract the digits (like 36 or 60)
# \d = any digit (0–9)
#+ = 1 or more times
#So \d+ = "grab all digits
#astype means we are converting the extracted string to int

In [ ]:
#Converting categorical columns to numeric using one-hot encoding
data_model =  pd.get_dummies(data_model,columns=['purpose', 'home_ownership'],drop_first=True)
# one-hot encoding creats 1/0 like yes/no column for each category, means if the loan was for credit card purpose it 1 or else 0
# then it will be 1 for other purpose if not credit card

In [ ]:
# split into features (X) and Target (y)
X= data_model.drop(target, axis=1)
y=data_model[target]
# Drop target to work on selected feature cleanly and axis=1 means working with the rest of columns and not dropping them
# we should not write like data_model[selected_feature] to use selected feature cause then its not manual and
# we have to keep 100% updated data
# the best practice in one-hot encoding is to drop the other column and let the data workds on its own, even if we add any feature in that

In [ ]:
print(X.shape)


(1345310, 25)


In [ ]:
print(y.shape)

(1345310,)


In [ ]:
y.head()
y.value_counts()

,count
loan_status,
0,1076751
1,268559


In [ ]:
X

,loan_amnt,term,int_rate,installment,annual_inc,dti,fico_range_high,purpose_credit_card,purpose_debt_consolidation,purpose_educational,...,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,3600.0,36,13.99,123.03,55000.0,5.91,679.0,False,True,False,...,False,False,False,False,False,True,False,False,False,False
1,24700.0,36,11.99,820.28,65000.0,16.06,719.0,False,False,False,...,False,False,True,False,False,True,False,False,False,False
2,20000.0,60,10.78,432.66,63000.0,10.78,699.0,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,10400.0,60,22.45,289.91,104433.0,25.37,699.0,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,11950.0,36,13.44,405.18,34000.0,10.20,694.0,False,True,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345305,18000.0,60,9.49,377.95,130000.0,20.59,739.0,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1345306,29400.0,60,13.99,683.94,180792.0,22.03,709.0,False,True,False,...,False,False,False,False,False,True,False,False,False,False
1345307,32000.0,60,14.49,752.74,157000.0,10.34,739.0,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1345308,16000.0,60,12.79,362.34,150000.0,12.25,669.0,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
# Model Train-Test Split
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(
    X,y,
    test_size = 0.2,
    random_state = 42,
    stratify= y #keeps same class ratio in train and test
)

In [ ]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train value counts:\n", y_train.value_counts(normalize=True))
print("y_test value counts:\n", y_test.value_counts(normalize=True))

X_train shape: (1076248, 25)
X_test shape: (269062, 25)
y_train value counts:
 loan_status
0    0.800374
1    0.199626
Name: proportion, dtype: float64
y_test value counts:
 loan_status
0    0.800373
1    0.199627
Name: proportion, dtype: float64


In [ ]:
# use logistic regression
# logistic regression is used because its a classification model
#  0 = Fully Paid
#❌ 1 = Charged Off (Defaulted)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

# scale the data- standardizing the data-ensures more stable training for models like LR
# cause some columns like lona amnt is in thousands, dti has 20-30,etc so stability is important
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# evaluating the model
y_pred = log_model.predict(X_test_scaled)

print("Accuracy", accuracy_score(y_test,y_pred))
print("Classificaion Report:\n", classification_report(y_test,y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test,y_pred))


Accuracy 0.8007187934379437
Classificaion Report:
               precision    recall  f1-score   support

           0       0.81      0.98      0.89    215350
           1       0.51      0.07      0.12     53712

    accuracy                           0.80    269062
   macro avg       0.66      0.53      0.51    269062
weighted avg       0.75      0.80      0.74    269062

Confusion Matrix:
 [[211661   3689]
 [ 49930   3782]]


In [ ]:
#logistic Regression is not giving us imbalanced prediction due to imbalanced data and is too simple for this real-world dataset
# so we are going to use Random Forest model which is more powerful and can handle imbalanced data better

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state = 42, n_jobs=-1)
rf_model.fit (X_train,y_train)

RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=42)

In [ ]:
y_pred = rf_model.predict(X_test)

print("accuracy:", accuracy_score(y_test,y_pred))
print("Classification Report:\n", classification_report (y_test,y_pred))
print("Confusin Matrix:\n", confusion_matrix(y_test,y_pred))

accuracy: 0.7971136764017215
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.97      0.88    215350
           1       0.46      0.09      0.15     53712

    accuracy                           0.80    269062
   macro avg       0.63      0.53      0.52    269062
weighted avg       0.74      0.80      0.74    269062

Confusin Matrix:
 [[209836   5514]
 [ 49075   4637]]


In [ ]:
# still we are not getting quite good predcitions results
# now we are gonna use XGBoost - which is powerful for tabular data

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
ratio = y_train.value_counts()[0] / y_train.value_counts()[1]


xgb_model=XGBClassifier (
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    eval_metric='logloss',
    scale_pos_weight=ratio,
    subsample=0.8,
    colsample_bytree=0.8
)
xgb_model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
y_pred=xgb_model.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))


Accuracy 0.6425173380113134
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.64      0.74    215350
           1       0.31      0.67      0.43     53712

    accuracy                           0.64    269062
   macro avg       0.60      0.65      0.58    269062
weighted avg       0.77      0.64      0.68    269062

Confusion Matrix:
 [[136990  78360]
 [ 17825  35887]]


In [ ]:
import joblib
joblib.dump(xgb_model,"loan_default_model.pkl")
joblib.dump(X.columns.tolist(), 'feature_names.pkl')

['feature_names.pkl']

In [ ]:
from google.colab import files
files.download('loan_default_model.pkl')
files.download('feature_names.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 134.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.5 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import joblib
import pandas as pd

In [ ]:
code """
import streamlit as st
import joblib
import pandas as pd

model = joblib.load('loan_default_model.pkl')
features = joblib.load('feature_names.pkl')

st.title("Loan Default Prediction App")
st.write ("Enter loan details to predict default Probability :")

user_input={}
for col in features:
  if "_" in col:
    user_input[col] = st.checkbox(col.replace('_', '').title())
  else:
    user_input[col] = st.number_input(col.replace('_', '').title(),value = 0.0)
  input_df=pd.DataFrame([user_input],columns = features)

  if st.button("Predict"):
    pred = model.predict(input_df)[0]
    prob = model.predict_proba(input_df)[0][1]
    st.markdown(f"## Prediction: **{'Default' if pred==1 else 'Fully Paid'}**")
    st.markdown(f"## Default Probability:{prob:.2%}")
    """
with open ("app.py", "w") as f:
  f.write(code)

  from google.colab import files
  files.download("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
!pip freeze > requirements.txt
from google.colab import files
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>